In [1]:
import numpy as np
from taskmanager import TaskManager

In [2]:
tm = TaskManager()
# ---------- #
# for _ in range(100):
#     tm.add_random_task()
# tm.complete()
# tm.save_tasks('benchmark_tasks.csv')
# --- OR --- #
tm.load_tasks('benchmark_tasks.csv')
tm.complete()
# ---------- #

In [3]:
results = tm.solve(chunk_size=6)
print('Number of equivalent solutions found:', len(results))
print('Penalty per task', results.penalty)
print('Cumulative required time', results.cumlength)
print('Ordered task indices', solution[..., 5])
print('Total penalty:', penalty.sum(axis=1))
print('Total length:', length.sum(axis=1))

Iterating initial heuristics...


  0%|          | 0/2 [00:00<?, ?it/s]

Solving and refining chunks...


100%|██████████| 17/17 [00:00<00:00, 74.82it/s]

Optimizing consecutive task types...


...

Solving and refining chunks...


100%|██████████| 17/17 [00:00<00:00, 83.90it/s] 


Optimizing consecutive task types...


100%|██████████| 2/2 [01:29<00:00, 44.97s/it]


TypeError: object of type 'Results' has no len()

In [ ]:
Total penalty: [225 225]
Total length: [353 353]

Total penalty: [155 155]
Total length: [355 355]